In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE209875"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE209875"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE209875.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE209875.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE209875.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
import os

# 1. First inspect the directory contents to identify what files are available
print("Available files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Since there's only one file, we'll use it for both SOFT and matrix data
if len(files) > 0:
    matrix_file = os.path.join(in_cohort_dir, files[0])
    soft_file = matrix_file  # Same file serves both purposes
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)

    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    raise FileNotFoundError("No files found in the directory")


Available files in the directory:
['GSE209875-GPL21263_series_matrix.txt.gz']

Background Information:
!Series_title	"MicroRNA and mRNA expression profiles of hepatobiliary tumor patients' tissue together with those of non-tumor part tissues and serum exosomes."
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"

Sample Characteristics Dictionary:
{0: ['histology: Cholangiocarcinoma, tumor part', 'histology: Cholangiocarcinoma (less than 40 generations), tumor part', 'histology: Cholangiocarcinoma, non-tumor part', 'histology: Cholangiocarcinoma (less than 40 generations), non-tumor part', 'histology: Hepatocellular carcinoma, tumor part', 'histology: Hepatocellular carcinoma, non-tumor part', 'histology: Inflammatory pseudotumor (benign), tumor part', 'histology: Angiomyolipoma (benign), tumor part', 'histology: Adenomyomatous hyperplasia of gallbladder (benign), tumor part', 'histology: Focal nodular hyper

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and sample characteristics, while this dataset contains mRNA data,
# the molecule field indicates this specific matrix file only contains miRNA data
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: The histology field (key 0) contains information about liver cancer types
trait_row = 0

# For age: Available in key 1
age_row = 1

# For gender: Available in key 2
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert histology data to binary trait indicating liver cancer (1) vs benign/non-tumor (0)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Categorize based on the histology description
    if 'benign' in value.lower() or 'non-tumor part' in value.lower():
        return 0  # Benign condition or non-tumor tissue
    elif 'carcinoma' in value.lower() and 'tumor part' in value.lower():
        return 1  # Malignant tumor tissue
    elif 'carcinoma' in value.lower() and 'tumor part' not in value.lower() and 'non-tumor part' not in value.lower():
        return 1  # Just specifying carcinoma without tissue part
    else:
        return None  # Unknown or unclear classification

def convert_age(value):
    """Convert age data to continuous value"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary (0=female, 1=male)"""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering based on gene and trait availability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Create a proper DataFrame from the sample characteristics data
    # We need to create a DataFrame where each column is a sample and rows are characteristics
    sample_chars = {
        0: ['histology: Cholangiocarcinoma, tumor part', 'histology: Cholangiocarcinoma (less than 40 generations), tumor part', 'histology: Cholangiocarcinoma, non-tumor part', 'histology: Cholangiocarcinoma (less than 40 generations), non-tumor part', 'histology: Hepatocellular carcinoma, tumor part', 'histology: Hepatocellular carcinoma, non-tumor part', 'histology: Inflammatory pseudotumor (benign), tumor part', 'histology: Angiomyolipoma (benign), tumor part', 'histology: Adenomyomatous hyperplasia of gallbladder (benign), tumor part', 'histology: Focal nodular hyperplasia of liver (benign), tumor part', 'histology: Inflammatory pseudotumor (benign), non-tumor part', 'histology: Angiomyolipoma (benign), non-tumor part', 'histology: Adenomyomatous hyperplasia of gallbladder (benign), non-tumor part', 'histology: Focal nodular hyperplasia of liver (benign), non-tumor part', 'histology: Cholangiocarcinoma', 'histology: Cholangiocarcinoma (less than 40 generations)', 'histology: Hepatocellular carcinoma', 'histology: Inflammatory pseudotumor (benign)', 'histology: Angiomyolipoma (benign)', 'histology: Adenomyomatous hyperplasia of gallbladder (benign)', 'histology: Focal nodular hyperplasia of liver (benign)'],
        1: ['age: 63', 'age: 34', 'age: 73', 'age: 76', 'age: 71', 'age: 68', 'age: 39', 'age: 31', 'age: 48', 'age: 66', 'age: 62', 'age: 75', 'age: 65', 'age: 55'],
        2: ['Sex: M', 'Sex: F'],
        3: ['molecule: miRNA']
    }
    
    # Create a DataFrame in the expected format (characteristics as rows) with sample IDs as columns
    # For simplicity, we'll create a mock format with sample IDs as GSM1, GSM2, etc.
    # First, determine how many samples we need (maximum number needed for any characteristic)
    max_samples = max(len(values) for values in sample_chars.values())
    
    # Create a DataFrame with rows for each characteristic type
    clinical_data = pd.DataFrame(index=[0, 1, 2, 3])
    
    # Add columns for each sample
    for i in range(max_samples):
        sample_id = f"GSM{i+1}"
        sample_data = {}
        
        # For each characteristic type, get the value if available
        for char_type in sample_chars:
            if i < len(sample_chars[char_type]):
                sample_data[char_type] = sample_chars[char_type][i]
            else:
                sample_data[char_type] = None
        
        clinical_data[sample_id] = pd.Series(sample_data)
    
    # Extract clinical features using the provided function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM1': [1.0, 63.0, 1.0], 'GSM2': [1.0, 34.0, 0.0], 'GSM3': [0.0, 73.0, nan], 'GSM4': [0.0, 76.0, nan], 'GSM5': [1.0, 71.0, nan], 'GSM6': [0.0, 68.0, nan], 'GSM7': [0.0, 39.0, nan], 'GSM8': [0.0, 31.0, nan], 'GSM9': [0.0, 48.0, nan], 'GSM10': [0.0, 66.0, nan], 'GSM11': [0.0, 62.0, nan], 'GSM12': [0.0, 75.0, nan], 'GSM13': [0.0, 65.0, nan], 'GSM14': [0.0, 55.0, nan], 'GSM15': [1.0, nan, nan], 'GSM16': [1.0, nan, nan], 'GSM17': [1.0, nan, nan], 'GSM18': [0.0, nan, nan], 'GSM19': [0.0, nan, nan], 'GSM20': [0.0, nan, nan], 'GSM21': [0.0, nan, nan]}
Clinical data saved to: ../../output/preprocess/Liver_Cancer/clinical_data/GSE209875.csv


### Step 3: Gene Data Extraction

In [4]:
# Check if the dataset contains gene expression data based on previous assessment
if not is_gene_available:
    print("This dataset does not contain gene expression data (only miRNA data).")
    print("Skipping gene expression data extraction.")
else:
    # Get the matrix file directly rather than using geo_get_relevant_filepaths
    files = os.listdir(in_cohort_dir)
    if len(files) > 0:
        matrix_file = os.path.join(in_cohort_dir, files[0])
        print(f"Matrix file found: {matrix_file}")
        
        try:
            # Extract gene data
            gene_data = get_genetic_data(matrix_file)
            print(f"Gene data shape: {gene_data.shape}")
            
            # Print the first 20 gene/probe identifiers
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        except Exception as e:
            print(f"Error extracting gene data: {e}")
    else:
        print("No files found in the input directory.")

This dataset does not contain gene expression data (only miRNA data).
Skipping gene expression data extraction.
